In [54]:
# If necessary, install packages
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
ERROR: No matching distribution found for os


In [3]:
!pip install gdown

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
# no longer in use, as data is no longer stored in the repo - see code below

# import pandas as pd

# # Importing data as DataFrames
# df_lobbying = pd.read_csv('data/lobbying/lob_lobbying.csv', header=None)
# df_industry = pd.read_csv('data/lobbying/lob_indus.csv', header=None)
# df_agency = pd.read_csv('data/lobbying/lob_agency.csv', header=None)
# df_bills = pd.read_csv('data/lobbying/lob_bills.csv', header=None)
# #df_issue = pd.read_csv('data/lobbying/lob_issue.csv', header=None)
# #df_lobbyist = pd.read_csv('data/lobbying/lob_lobbyist.csv', header=None)

# # Applying correct column name. 
# df_lobbying.columns = ['UniqID','Registrant_Raw','Registrant','IsFirm','Client_raw','Client','UItorg','Amount','Catcode','Source','Self','IncludeNSFS','Use','Ind','Year','Type','Typelong','Type']
# df_industry.columns = ['Client','Sub','Total','Year','Catcode']
# df_agency.columns = ['UniqID','AgencyID','Agency']
# df_bills.columns = ['B_ID', 'SI_ID', 'CongNo', 'Bill_Name']
# #df_issue.columns = ['SI_ID','UniqID','IssueID','Issue','SpecificIssue','Year']
# #df_lobbyist.columns = ['UniqID','Lobbyist_raw','Lobbyist','LobbyistID','Year','OfficialPosition','CID','Formercongmem']

In [2]:
# file IDs and table namesfor lobbying data files from Google Drive

lob_rpt = ("1CIq7XwaFTJmnUKdAvYyh53Et2XSJRX9d", "lob_rpt")
lob_lobbyist = ("1q1ZyLvUhsxsbPLdi_CmrNH0FqLoorr6p", "lob_lobbyist")
lob_lobbying = ("19Jef89spXbkExNpYA-s_e8GRii-0tReu", "lob_lobbying")
lob_issue = ("18-D_vW4dAHJlvFs4ARK4lgx6KdsAXWoq", "lob_issue")
lob_indus = ("1yldZYLcZTLIMQ5Xa-e-ecGgQGFWv5n4A", "lob_indus")
lob_bills = ("1qqQvNXtdhyI8KYcctpVUZ3FoIwvjWaG1", "lob_bills")
lob_agency = ("1CFi0Itpi_qg4X4wicg1FIg8S_FJfvCN0", "lob_agency")
CatCodeReference = ("1vfVA-v-lUMuykF9efl2oDN5DJK2t1PwA", "CatCodeReference")
CandidateIds = ("1k2lhAcuSQbn5z7NN3USPEbX_Rg5VV_Ky", "CandidateIds")
congress_bill_data_id = ("1-MnzV0iZ2FPYiuQgOmggjmXvxGmBzYV8", "congress_bill_data")

In [4]:
import pandas as pd
import gdown
import os

def load_csv(table, save_file=False, file_path=None):
    """
    Downloads a large CSV file from Google Drive into a Pandas DataFrame.
    
    Parameters:
        file_info (tuple): A tuple containing the Google Drive file ID and the table name.
        save_file (bool): If True, the file is saved. If False, it is deleted after loading.
        file_path (str, optional): The directory path where the file should be saved. Defaults to current directory.

    Returns:
        pd.DataFrame: The loaded Pandas DataFrame.
    """
    file_id, table_name = table
    
    # Set filename as the table name
    temp_file = f"{table_name}.csv"
    
    # If a file path is provided, join it with the filename
    if file_path:
        temp_file = os.path.join(file_path, temp_file)

    # Construct Google Drive download URL
    url = f"https://drive.google.com/uc?export=download&id={file_id}"

    # Download the file
    gdown.download(url, temp_file, quiet=False)

    # Load the CSV file into a Pandas DataFrame
    df = pd.read_csv(temp_file,header=None)

    # Remove the file if save_file is False
    if not save_file:
        os.remove(temp_file)

    return df

/Users/kristianmadslangrud/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [57]:
# Example usage:
df = load_csv(lob_rpt, save_file=False)  # Specify file_path if needed
print(df.head())

Downloading...
From: https://drive.google.com/uc?export=download&id=1CIq7XwaFTJmnUKdAvYyh53Et2XSJRX9d
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_rpt.csv
100%|██████████| 3.29k/3.29k [00:00<00:00, 406kB/s]

                                  0    1
0                   MID-YEAR REPORT    m
1                MID-YEAR AMENDMENT   ma
2            MID-YEAR (NO ACTIVITY)   mn
3  MID-YEAR AMENDMENT (NO ACTIVITY)  man
4              MID-YEAR TERMINATION   mt


## Data Cleaning

In [58]:
# Creating function to visualize key aspects of each dataframe. 
def glimpse(df):
    print(f'\nShape of dataframe is: {df.shape}\n')
    print(f'Dataframe has the following columns:\n{df.columns}\n')
    print(f'Dataframe has the following data types:\n{df.dtypes}\n')
    print(f'Overview of missing values:\n{df.isnull().sum()}\n')
    print(f'Missing values as percentage of total:\n{(df.isnull().sum()/len(df))*100}\n')
    print(f'First 5 Columns:{df.head()}\n')
    print(f'Last 5 Columns:{df.tail()}\n')

### Lobbying Data

In [59]:
df_lobbying = load_csv(lob_lobbying, save_file = False)
df_lobbying.columns = ['UniqID','Registrant_Raw','Registrant','IsLobbyingFirm','Client_raw','Client','UItorg','Amount','Catcode','Source','Self','IncludeNSFS','Use','Ind','Year','Type','Typelong','Affiliate']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=19Jef89spXbkExNpYA-s_e8GRii-0tReu
From (redirected): https://drive.google.com/uc?export=download&id=19Jef89spXbkExNpYA-s_e8GRii-0tReu&confirm=t&uuid=43745d74-284d-45b6-9ce7-57234f99ffa9
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_lobbying.csv
100%|██████████| 351M/351M [00:23<00:00, 14.7MB/s] 
/var/folders/pg/ghgx2xgn3qb65km4qhtfgzhr0000gn/T/ipykernel_56559/2418707545.py:33: DtypeWarning: Columns (7,11,14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(temp_file,header=None)


In [60]:
glimpse(df_lobbying)


Shape of dataframe is: (1544073, 18)

Dataframe has the following columns:
Index(['UniqID', 'Registrant_Raw', 'Registrant', 'IsLobbyingFirm',
       'Client_raw', 'Client', 'UItorg', 'Amount', 'Catcode', 'Source', 'Self',
       'IncludeNSFS', 'Use', 'Ind', 'Year', 'Type', 'Typelong', 'Affiliate'],
      dtype='object')

Dataframe has the following data types:
UniqID            object
Registrant_Raw    object
Registrant        object
IsLobbyingFirm    object
Client_raw        object
Client            object
UItorg            object
Amount            object
Catcode           object
Source            object
Self              object
IncludeNSFS       object
Use               object
Ind               object
Year              object
Type              object
Typelong          object
Affiliate         object
dtype: object

Overview of missing values:
UniqID                 20
Registrant_Raw         49
Registrant             58
IsLobbyingFirm     229072
Client_raw            396
Client       

In [61]:
#Dropping unecessary columns
df_lobbying = df_lobbying.drop(
['Registrant_Raw', #Same as Registrant
 'Client_raw', # Same as Client
 'Source', # Irrelevant to analysis
 'Typelong' # Same as Type
], axis=1)

# Removing all records from lobbying dataframe without an ID
df_lobbying = df_lobbying.dropna(subset=['UniqID'])

# Capitalizing all inputs in Catcode column
df_lobbying['Catcode'] = df_lobbying['Catcode'].str.upper()

# Replacing missing values in selected boolean columns with 'n' column with 'n'
boolean_columns = ['IsLobbyingFirm','Affiliate','IncludeNSFS','Ind']
df_lobbying[boolean_columns] = df_lobbying[boolean_columns].fillna(value='n')

# Changing datatype of columns
df_lobbying['Amount'] = pd.to_numeric(df_lobbying['Amount'], errors='coerce')
df_lobbying = df_lobbying.dropna(subset=['Amount'])
df_lobbying['Year'] = df_lobbying['Year'].astype('int')
df_lobbying[boolean_columns] = df_lobbying[boolean_columns].apply(lambda col: col.map({'y': True, 'n': False}).astype(bool))

# Dropping all lobbying records with Use == n, as these reports have been amendent/updated and a more recent report is avaliable.  
df_lobbying = df_lobbying[df_lobbying['Use'] != 'n']
df_lobbying = df_lobbying.drop('Use',axis=1)

# Only including records after 2011.
df_lobbying = df_lobbying[df_lobbying['Year'] >= 2011]
df_lobbying = df_lobbying.drop('Affiliate',axis=1) # Dropping the Affiliate column as all records are equal to False after Slicing. 


In [63]:
!pip install ace-tools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [76]:
# Extract quarter dynamically (first character + "Q")
df_lobbying['Quarter'] = 'Q' + df_lobbying['Type'].str.extract(r'(\d)')[0]

# Extract report type based on suffix in 'Type' column
df_lobbying['ReportType'] = df_lobbying['Type'].apply(lambda x: 
    'Standard Report' if len(x) == 2 else
    'Amendment' if x.endswith("a") else
    'Amendment (No Activity)' if x.endswith('an') else
    'No Activity' if x.endswith('n') else
    'Termination' if x.endswith('t') else
    "Termination Amendment" if x.endswith('ta') else
    'Termination (No Activity)' if x.endswith('tn') else
    'Unknown')

df_lobbying.drop('Type',axis=1) # Dropping 'Type' column as it is uneccessary now. 

/var/folders/pg/ghgx2xgn3qb65km4qhtfgzhr0000gn/T/ipykernel_56559/4105425718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lobbying['Quarter'] = 'Q' + df_lobbying['Type'].str.extract(r'(\d)')[0]
/var/folders/pg/ghgx2xgn3qb65km4qhtfgzhr0000gn/T/ipykernel_56559/4105425718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lobbying['ReportType'] = df_lobbying['Type'].apply(lambda x:


,UniqID,Registrant,IsLobbyingFirm,Client,UItorg,Amount,Catcode,Self,IncludeNSFS,Ind,Year,Quarter,ReportType
0,82c5f661-a637-45ad-a3a6-b5ba18cf8962,AstraZeneca Pharmaceuticals,False,AstraZeneca Pharmaceuticals,AstraZeneca PLC,1370000.0,H4300,x,False,True,2021,Q4,Amendment
4,87ff989d-9d12-4fef-84ef-ab69cd616894,Financial Executives International,False,Financial Executives International,Financial Executives International,21650.0,F5000,p,False,True,2021,Q4,Standard Report
5,88bac485-00bb-4915-8ca5-8d0ec589ec28,Capitol Hill Consulting Group,True,Association for Accessible Medicines,Association for Accessible Medicines,40000.0,H4300,i,False,False,2021,Q4,Standard Report
6,8902b3fe-2dde-440a-a60d-c57b1a212ece,Revere Federal Strategies,True,Accion Systems,Accion Systems,12000.0,Y4000,n,False,True,2021,Q4,Standard Report
8,89ca2bea-03f3-4e46-b005-303c36827bfb,First Principles Strategies,True,Exela Pharma Sciences,Exela Pharma Sciences,20000.0,H4300,n,False,True,2021,Q4,Termination
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544051,8b4e4648-720c-4da2-ba96-ee9e5733e76d,Navigators Global,True,Affordable Equity Partners,Affordable Equity Partners,30000.0,F4700,n,False,True,2022,Q4,Standard Report
1544057,7043c37e-e454-4c7f-9677-31ae8dff650a,Cozen O'Connor,True,"Withum, Smith & Brown","Withum, Smith & Brown",50000.0,F5100,n,False,True,2023,Q1,Standard Report
1544066,12a47a18-11f4-43b2-bb1f-3db51e07306b,P Street Project,False,P Street Project,Progressive Change Campaign Cmte,20000.0,J1200,x,False,True,2022,Q4,Standard Report
1544068,0854dd96-ed91-4219-99d9-b84bf06ee2f8,American Express,False,American Express,American Express,440000.0,F1400,p,False,True,2022,Q4,Standard Report


### Industry

In [12]:
df_industry = load_csv(lob_indus, save_file = False)
df_industry.columns = ['Client','Sub','Total','Year','Catcode']

Downloading...
From: https://drive.google.com/uc?export=download&id=1yldZYLcZTLIMQ5Xa-e-ecGgQGFWv5n4A
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_indus.csv
100%|██████████| 20.3M/20.3M [00:00<00:00, 22.5MB/s]


In [ ]:
glimpse(df_industry)


Shape of dataframe is: (309730, 5)

Dataframe has the following columns:
Index(['Client', 'Sub', 'Total', 'Year', 'Catcode'], dtype='object')

Dataframe has the following data types:
Client     object
Sub        object
Total       int64
Year        int64
Catcode    object
dtype: object

Overview of missing values:
Client     0
Sub        0
Total      0
Year       0
Catcode    0
dtype: int64

Missing values as percentage of total:
Client     0.0
Sub        0.0
Total      0.0
Year       0.0
Catcode    0.0
dtype: float64

First 5 Columns:                                              Client  \
0  National Assn for County Community & Econ Develop   
1                       Fox Valley Technical College   
2            Employers Cncl on Flexible Compensation   
3            American Federation of TV/Radio Artists   
4                     Visiting Nurse Assn of America   

                                                 Sub   Total  Year Catcode  
0  National Assn for County Community & Econ

In [14]:
# Only including records after 2011.
df_industry = df_industry[df_industry['Year'] >= 2011]

# Capitalizing all inputs in Catcode column
df_industry['Catcode'] = df_industry['Catcode'].str.upper()

#### Catcode References

In [15]:
# Try reading the file with extra handling for irregularities
df_catcode_references = load_csv(CatCodeReference, save_file = False)
df_catcode_references.columns = ['Catcode','Catname','Catorder','Industry','Sector','SectorLong']

Downloading...
From: https://drive.google.com/uc?export=download&id=1vfVA-v-lUMuykF9efl2oDN5DJK2t1PwA
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/CatCodeReference.csv
100%|██████████| 43.0k/43.0k [00:00<00:00, 879kB/s]


In [16]:
# merging catcode reference with industry column
df_industry = pd.merge(df_industry, df_catcode_references[['Catcode','Catname', 'Industry', 'Sector']],
                     left_on='Catcode', right_on='Catcode', how='left')

### Agency

In [17]:
df_agency = load_csv(lob_agency, save_file=False)
df_agency.columns = ['UniqID','AgencyID','Agency']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1CFi0Itpi_qg4X4wicg1FIg8S_FJfvCN0
From (redirected): https://drive.google.com/uc?export=download&id=1CFi0Itpi_qg4X4wicg1FIg8S_FJfvCN0&confirm=t&uuid=561ecda0-96ac-4cb0-9a03-3ab2efa617fc
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_agency.csv
100%|██████████| 256M/256M [00:07<00:00, 34.7MB/s] 


In [18]:
glimpse(df_agency)


Shape of dataframe is: (4097763, 3)

Dataframe has the following columns:
Index(['UniqID', 'AgencyID', 'Agency'], dtype='object')

Dataframe has the following data types:
UniqID      object
AgencyID     int64
Agency      object
dtype: object

Overview of missing values:
UniqID      0
AgencyID    0
Agency      0
dtype: int64

Missing values as percentage of total:
UniqID      0.0
AgencyID    0.0
Agency      0.0
dtype: float64

First 5 Columns:                                 UniqID  AgencyID  \
0  BB7367A7-7B60-4DED-AA2D-A94771A9EBE8         1   
1  BB7367A7-7B60-4DED-AA2D-A94771A9EBE8         2   
2  04366C6F-B0CE-4C28-87BF-EE1CC8A9BB41         2   
3  04366C6F-B0CE-4C28-87BF-EE1CC8A9BB41        34   
4  04366C6F-B0CE-4C28-87BF-EE1CC8A9BB41       134   

                                     Agency  
0                                 US Senate  
1               US House of Representatives  
2               US House of Representatives  
3           Dept of Health & Human Services  
4  C

### Issues

In [9]:
df_issue = load_csv(lob_issue, save_file=False)
df_issue.columns = ['SI_ID','UniqID','IssueID','Issue','SpecificIssue','Year']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=18-D_vW4dAHJlvFs4ARK4lgx6KdsAXWoq
From (redirected): https://drive.google.com/uc?export=download&id=18-D_vW4dAHJlvFs4ARK4lgx6KdsAXWoq&confirm=t&uuid=b2daf66b-9a8b-4c15-9f14-8df1bffa2e23
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_issue.csv
100%|██████████| 787M/787M [00:19<00:00, 40.6MB/s]]


In [ ]:
glimpse(df_issue)


Shape of dataframe is: (3009864, 6)

Dataframe has the following columns:
Index(['SI_ID', 'UniqID', 'IssueID', 'Issue', 'SpecificIssue', 'Year'], dtype='object')

Dataframe has the following data types:
SI_ID             int64
UniqID           object
IssueID          object
Issue            object
SpecificIssue    object
Year              int64
dtype: object

Overview of missing values:
SI_ID                 0
UniqID                0
IssueID               0
Issue                 0
SpecificIssue    495488
Year                  0
dtype: int64

Missing values as percentage of total:
SI_ID             0.000000
UniqID            0.000000
IssueID           0.000000
Issue             0.000000
SpecificIssue    16.462139
Year              0.000000
dtype: float64

First 5 Columns:     SI_ID                                UniqID IssueID  \
0  3001624  02e92bd6-0159-495e-9d00-8a490a0be8be     DIS   
1  3001625  02e92bd6-0159-495e-9d00-8a490a0be8be     ENV   
2  3001626  02e92bd6-0159-495e-9d00-8a

In [21]:
# Only including records after 2011.
df_issue = df_issue[df_issue['Year'] >= 2011]

# Retrieving records with missing values for SpecificIssue 
specific_issue_missing = df_issue[df_issue['SpecificIssue'].isna()]

# Imputing missing values in SpecificIssue column with value from Issue column. 
df_issue['SpecificIssue'] = df_issue['SpecificIssue'].fillna(df_issue['Issue'])

#### Bills

In [26]:
df_bills = load_csv(lob_bills, save_file=False)
df_bills.columns = ['B_ID', 'SI_ID', 'CongNo', 'Bill_Name']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1qqQvNXtdhyI8KYcctpVUZ3FoIwvjWaG1
From (redirected): https://drive.google.com/uc?export=download&id=1qqQvNXtdhyI8KYcctpVUZ3FoIwvjWaG1&confirm=t&uuid=c8623fe3-13c8-40f6-80aa-331e597a5afd
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_bills.csv
100%|██████████| 219M/219M [00:05<00:00, 36.9MB/s] 


In [6]:
glimpse(df_bills)

NameError: name 'glimpse' is not defined

In [ ]:
# Importing Year column from the issues dataframe 
# this is to impute missing values for congress number by cross-referencing Year with a dictionary containing Year and Congress No. at the time. 
df_bills = df_bills.merge(df_issue[['SI_ID', 'Year']], on='SI_ID', how='left')

# Dictionary mapping Congress by year. Retrieved from: https://www.congress.gov/help/field-values/congresses
congress_number_by_year = {
2025: 119.0, 2026: 119.0,2023: 118.0,2024: 118.0,2021: 117.0,2022: 117.0,2019: 116.0,2020: 116.0,
2017: 115.0,2018: 115.0,2015: 114.0,2016: 114.0,2013: 113.0,2014: 113.0,2011: 112.0,
2012: 112.0,2009: 111.0,2010: 111.0,2007: 110.0,2008: 110.0,2005: 109.0,2006: 109.0,
2003: 108.0,2004: 108.0,2001: 107.0,2002: 107.0,1999: 106.0,2000: 106.0,1998: 105.0
}

# Impute missing values
df_bills['CongNo'] = df_bills['CongNo'].fillna(df_bills['Year'].map(congress_number_by_year))

# Dropping 'Year' column again
df_bills = df_bills.drop('Year',axis=1)

# Converting datatype of CongNo column
df_bills['CongNo'] = df_bills['CongNo'].astype('Int64')

In [28]:
# Dropping records with missing value in CongNo, as these records are not connected to Lobby Issues dataframe through Foreign Key - and are hence excessive
df_bills = df_bills.dropna(subset=['CongNo'])

In [29]:
df_bills["Bill_Name"] = df_bills["Bill_Name"].str.replace(r"^H\.?(\d+)$", r"H.R.\1", regex=True)

#### Importing additional information about bills

In [30]:
df_congress_bill_data = load_csv(congress_bill_data_id, save_file=False)
df_congress_bill_data.columns = ['bill_number','congress_number','bill_title','policy_area','introduced_date',
                                 'bill_type','full_bill_code','sponsor_name','sponsor_party','sponsor_bioguide_id']
df_congress_bill_data = df_congress_bill_data.iloc[1:] # dropping first row of dataframe



Downloading...
From: https://drive.google.com/uc?export=download&id=1-MnzV0iZ2FPYiuQgOmggjmXvxGmBzYV8
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/congress_bill_data.csv
100%|██████████| 4.26M/4.26M [00:00<00:00, 8.07MB/s]


In [36]:
df_congress_bill_data.dtypes
df_congress_bill_data['congress_number'] = df_congress_bill_data['congress_number'].astype('Int64')

In [37]:
df_congress_bill_data = df_congress_bill_data.merge(
    df_bills[['B_ID', 'CongNo', 'Bill_Name']],
    left_on=['full_bill_code', 'congress_number'],
    right_on=['Bill_Name', 'CongNo'],
    how='left'
).drop(columns=['Bill_Name', 'CongNo'])  # Drop duplicate columns after merge if needed



In [ ]:
# Converting datatype of CongNo column
df_bills['CongNo'] = df_bills['CongNo'].astype('Int64')

### Lobbyist

In [ ]:
df_lobbyist = load_csv(lob_lobbyist,save_file=False)
df_lobbyist.columns = ['UniqID','Lobbyist_raw','Lobbyist','LobbyistID','Year','OfficialPosition','CID','Formercongmem']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1q1ZyLvUhsxsbPLdi_CmrNH0FqLoorr6p
From (redirected): https://drive.google.com/uc?export=download&id=1q1ZyLvUhsxsbPLdi_CmrNH0FqLoorr6p&confirm=t&uuid=19955945-0d51-49d8-a713-1199693831b7
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_lobbyist.csv
100%|██████████| 410M/410M [00:10<00:00, 37.9MB/s] 


In [27]:
glimpse(df_lobbyist)


Shape of dataframe is: (3497113, 8)

Dataframe has the following columns:
Index(['UniqID', 'Lobbyist_raw', 'Lobbyist', 'LobbyistID', 'Year',
       'OfficialPosition', 'CID', 'Formercongmem'],
      dtype='object')

Dataframe has the following data types:
UniqID               object
Lobbyist_raw         object
Lobbyist             object
LobbyistID           object
Year                float64
OfficialPosition     object
CID                  object
Formercongmem        object
dtype: object

Overview of missing values:
UniqID                  225
Lobbyist_raw          21873
Lobbyist              13953
LobbyistID            21952
Year                  21953
OfficialPosition    2819840
CID                 3403564
Formercongmem         29950
dtype: int64

Missing values as percentage of total:
UniqID               0.006434
Lobbyist_raw         0.625459
Lobbyist             0.398986
LobbyistID           0.627718
Year                 0.627746
OfficialPosition    80.633368
CID                

In [28]:
# Only including records after 2011.
df_lobbyist = df_lobbyist[df_lobbyist['Year'] >= 2011]

# Dropping missing values
df_lobbyist = df_lobbyist.dropna(subset=['LobbyistID'])
df_lobbyist = df_lobbyist.dropna(subset=['Year']) # Dropping this record as well as it does not have any UniqId

# Dropping uneccessary column
df_lobbyist = df_lobbyist.drop('Lobbyist_raw',axis=1)

# Chaging data type of Year column
df_lobbyist['Year'] = df_lobbyist['Year'].astype('Int64')

# Imputing missing values in 'Formercongmem' column -- if CID = Blank --> 'n', if CID != Blank --> 'y'
df_lobbyist.loc[df_lobbyist['Formercongmem'].isna(), 'Formercongmem'] = df_lobbyist['CID'].apply(
    lambda x: 'n' if pd.isna(x) else 'y')

# Changing data type of Formercongmem column to Boolean
df_lobbyist['Formercongmem'] = df_lobbyist['Formercongmem'].map({'y': True, 'n': False}).astype(bool)

# Imputing missing values for CID column
df_lobbyist['CID'] = df_lobbyist['CID'].fillna(value='Not Congress Member')


In [29]:
# Pringint unique values in official position column
print(len(df_lobbyist['OfficialPosition'].unique()))

# Considering the amount of missing values and unique values - OfficialPosition column is dropped due to providing small value to further analysis
df_lobbyist = df_lobbyist.drop('OfficialPosition',axis=1)

56604


#### Candidate IDs

In [30]:
df_candidate = load_csv(CandidateIds,save_file=False)
df_candidate.columns = ['CID', 'CRPName', 'Party', 'DistIDRunFor', 'FECCandID']

Downloading...
From: https://drive.google.com/uc?export=download&id=1k2lhAcuSQbn5z7NN3USPEbX_Rg5VV_Ky
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/CandidateIds.csv
100%|██████████| 970k/970k [00:00<00:00, 3.22MB/s]


In [31]:
# Merge on the common 'CID' column to include the columns from df_candidate
df_lobbyist = pd.merge(
    df_lobbyist,
    df_candidate[['CID', 'CRPName', 'Party', 'DistIDRunFor', 'FECCandID']],
    on='CID',
    how='left'
)

## Narrowing Scope

In [70]:
# Only including lobbying reports related to specific issues
chosen_issues = ['Health Issues', 'Medical Research & Clin Labs', 'Pharmacy', 'Medicare & Medicaid', 'Science & Technology', 'Taxes']
df_issue = df_issue[df_issue['Issue'].isin(chosen_issues)]

In [71]:
# Dropping lobbying reports that are no longer referenced in df_issue table
df_lobbying = df_lobbying[df_lobbying['UniqID'].isin(df_issue['UniqID'])]

# dropping lobbyist records for lobbyists that are no longer involved in the lobbying reports from cleaned df_lobbying df.
df_lobbyist = df_lobbyist[df_lobbyist['UniqID'].isin(df_lobbying['UniqID'])]

# droppping bills records that are not related to any of the chosen issues. 
df_bills = df_bills[df_bills['SI_ID'].isin(df_issue['SI_ID'])]

# droppping agency records that are not related to any of the current reports. 
df_agency = df_agency[df_agency['UniqID'].isin(df_lobbying['UniqID'])]

# droppping industry records that are not related to any of the current reports. --> How can we limit this further?
df_industry = df_industry[df_industry['Catcode'].isin(df_lobbying['Catcode'])]
